# Imports

In [1]:
import logging
import requests

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options

import tkinter as tk
from tkinter import messagebox

# CAPTCHA Solving Function

In [2]:
def solve_captcha(api_key, captcha_image_url):
    # Send CAPTCHA to 2Captcha
    response = requests.post(
        'http://2captcha.com/in.php',
        data={
            'key': api_key,
            'method': 'base64',
            'body': captcha_image_url,
            'json': 1
        }
    )
    request_id = response.json().get('request')

    # Wait for the CAPTCHA to be solved
    time.sleep(20)

    # Retrieve the CAPTCHA solution
    response = requests.get(
        'http://2captcha.com/res.php',
        params={
            'key': api_key,
            'action': 'get',
            'id': request_id,
            'json': 1
        }
    )
    return response.json().get('request')

# Automation Function

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def run_bot(username, password, root, api_key):
    options = Options()
    options.add_argument("--user-data-dir=C:/Users/user/AppData/Local/Temp/EdgeProfile")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-extensions")

    driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)
    try:
        logging.info("Navigating to the login page")
        driver.get("https://target_website.com")

        # LOGIN
        logging.info("Filling in login details")
        username_field = driver.find_element(By.ID, "username")
        password_field = driver.find_element(By.ID, "password")
        login_button = driver.find_element(By.ID, "login_button")
        username_field.send_keys(username)
        password_field.send_keys(password)
        login_button.click()

        # Accept Terms and Conditions
        logging.info("Accepting terms and conditions")
        terms_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "terms_button"))
        )
        terms_button.click()

        # Handle CAPTCHA
        logging.info("Solving CAPTCHA")
        captcha_image = driver.find_element(By.ID, "captcha_image")
        captcha_solution = solve_captcha(api_key, captcha_image.get_attribute("src"))
        captcha_input = driver.find_element(By.ID, "captcha_input")
        captcha_input.send_keys(captcha_solution)
        captcha_submit = driver.find_element(By.ID, "captcha_submit")
        captcha_submit.click()

        # Sidebar Menu
        wait = WebDriverWait(driver, 10)
        sidebar_menu = wait.until(EC.presence_of_element_located((By.ID, "sidebar_menu")))
        sidebar_toggle = sidebar_menu.find_element(By.ID, "sidebar_toggle")
        sidebar_toggle.click()

        sidebar_item = wait.until(EC.element_to_be_clickable((By.XPATH, "sidebar_item")))
        sidebar_item.click()

        confirmation_button = wait.until(EC.element_to_be_clickable((By.ID, "confirmation_button")))
        confirmation_button.click()
    except Exception as e:
        logging.error(f"An error occurred: {e}")
    finally:
        driver.quit()
        root.destroy()

# GUI

In [4]:
def main():
    root = tk.Tk()
    root.geometry("1000x1000")
    root.title("GUI")

    tk.Label(root, text="Enter your username:").pack()
    username_entry = tk.Entry(root)
    username_entry.pack()

    tk.Label(root, text="Enter your password:").pack()
    password_entry = tk.Entry(root, show='*')
    password_entry.pack()

    tk.Label(root, text="Enter your 2Captcha API key:").pack()
    api_key_entry = tk.Entry(root)
    api_key_entry.pack()

    disclaimer_label = tk.Label(root, text="text", fg="black", wraplength=480, justify=tk.LEFT, font=("Arial", 12))
    disclaimer_label.pack(pady=10)

    def on_submit():
        username = username_entry.get()
        password = password_entry.get()
        api_key = api_key_entry.get()

        if not username or not password or not api_key:
            messagebox.showerror("Error", "All fields are required!")
            return

    submit_button = tk.Button(root, text="Submit", command=on_submit)
    submit_button.pack()
    root.mainloop()

In [5]:
if __name__ == "__main__":
    main()